In [5]:
import pandas as pd 






In [6]:
# loading the data
file_path = r'D:\This PC\Documents\Data_Analysis\HUJI_course\projects_and_datasets\final_project\coffee_shop_sales_data\coffee_shop_sales.csv'
df = pd.read_csv(file_path)



# date & and time type convertion
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

## inspecting the data: 

### general inspection:

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149116 entries, 0 to 149115
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   transaction_id    149116 non-null  int64         
 1   transaction_date  149116 non-null  datetime64[ns]
 2   transaction_time  149116 non-null  object        
 3   transaction_qty   149116 non-null  int64         
 4   store_id          149116 non-null  int64         
 5   store_location    149116 non-null  object        
 6   product_id        149116 non-null  int64         
 7   unit_price        149116 non-null  float64       
 8   product_category  149116 non-null  object        
 9   product_type      149116 non-null  object        
 10  product_detail    149116 non-null  object        
 11  revenue           149116 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(5)
memory usage: 13.7+ MB


In [22]:
# cheking if there are any missing data (Nulls)
df.isnull().sum()

transaction_id      0
transaction_date    0
transaction_time    0
transaction_qty     0
store_id            0
store_location      0
product_id          0
unit_price          0
product_category    0
product_type        0
product_detail      0
revenue             0
dtype: int64

In [21]:
# checking for duplicate rows: 
print(f" number of duplicate rows: {df['transaction_id'].duplicated().sum()}")

 number of duplicate rows: 0


### quantity, prices and revenue: 

In [16]:
df[['transaction_qty', 'unit_price', 'revenue']].describe()

,transaction_qty,unit_price,revenue
count,149116.000000,149116.000000,149116.000000
mean,1.438276,3.382219,4.686367
std,0.542509,2.658723,4.227099
min,1.000000,0.800000,0.800000
25%,1.000000,2.500000,3.000000
50%,1.000000,3.000000,3.750000
75%,2.000000,3.750000,6.000000
max,8.000000,45.000000,360.000000


In [17]:
# checking if transaction quantity and unit price are ever 0 or negative: 
df.query('transaction_qty <= 0 or unit_price <= 0')[['transaction_id', 'unit_price']]

,transaction_id,unit_price


In [9]:
# checking which product is the one with max price 
df.query('unit_price == 45')[['product_id','product_category']]

,product_id,product_category
5187,8,Coffee beans
5879,8,Coffee beans
5944,8,Coffee beans
7626,8,Coffee beans
8452,8,Coffee beans
...,...,...
144743,8,Coffee beans
145411,8,Coffee beans
148573,8,Coffee beans
148702,8,Coffee beans


### date and time

In [12]:
# sampling the date and time columns:
df[['transaction_date', 'transaction_time']].head(5) 

,transaction_date,transaction_time
0,2023-01-01,07:06:11
1,2023-01-01,07:08:56
2,2023-01-01,07:14:04
3,2023-01-01,07:20:24
4,2023-01-01,07:22:41


In [15]:
# checking the time report range of the data 
print(f"first day of the report period: {df['transaction_date'].min()}")
print(f"last day of the report period: {df['transaction_date'].max()}")
      


first day of the report period: 2023-01-01 00:00:00
last day of the report period: 2023-06-30 00:00:00


### products and categories

In [18]:
# checking the unique product categories we have: 
print(f"number of unique product categories: {df['product_category'].nunique()}")
print(df['product_category'].unique())

number of unique product categories: 9
['Coffee' 'Tea' 'Drinking Chocolate' 'Bakery' 'Flavours' 'Loose Tea'
 'Coffee beans' 'Packaged Chocolate' 'Branded']


### stores and locations: 

In [23]:
# checking the distribution of transactions across the different store locations
df['store_location'].value_counts()

store_location
Hell's Kitchen     50735
Astoria            50599
Lower Manhattan    47782
Name: count, dtype: int64